In [13]:
pip install scipy numpy rdkit-pypi scikit-learn xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 1.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import bioalerts
import numpy as np
from bioalerts import LoadMolecules, Alerts, FPCalculator

In [2]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
# Machine learning modules
import sklearn
from sklearn import preprocessing
import scipy
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pandas as pd

In [3]:
from os import listdir
listdir('.')

['tutorial',
 '.git',
 'Untitled1.ipynb',
 'incorrect_molecules.csv',
 'scheme.svg',
 'README.md',
 'bioalerts',
 'Untitled.ipynb',
 'bioalterts test.ipynb',
 '.ipynb_checkpoints']

In [4]:


molecules = bioalerts.LoadMolecules.LoadMolecules("tutorial/datasets/Young_2002.sdf",
                                                  verbose=False)
molecules.ReadMolecules()
print ("Total number of input molecules: ", len(molecules.mols))



Format of the structures file = SDF
265 molecules (starting at zero) could not be processed.

This information has been saved in the following file: incorrect_molecules.csv

NOTE: the indexes of the molecules start at zero. Thus the first molecule is molecule 0.
Total number of input molecules:  1753


[14:26:40] Explicit valence for atom # 0 N, 4, is greater than permitted
[14:26:40] ERROR: Could not sanitize molecule ending on line 25
[14:26:40] ERROR: Explicit valence for atom # 0 N, 4, is greater than permitted
[14:26:40] Explicit valence for atom # 0 N, 4, is greater than permitted
[14:26:40] ERROR: Could not sanitize molecule ending on line 51
[14:26:40] ERROR: Explicit valence for atom # 0 N, 4, is greater than permitted
[14:26:40] Explicit valence for atom # 1 N, 4, is greater than permitted
[14:26:40] ERROR: Could not sanitize molecule ending on line 77
[14:26:40] ERROR: Explicit valence for atom # 1 N, 4, is greater than permitted
[14:26:40] Explicit valence for atom # 0 N, 4, is greater than permitted
[14:26:40] ERROR: Could not sanitize molecule ending on line 131
[14:26:40] ERROR: Explicit valence for atom # 0 N, 4, is greater than permitted
[14:26:40] Explicit valence for atom # 8 N, 4, is greater than permitted
[14:26:40] ERROR: Could not sanitize molecule ending on li

In [5]:
stride = int(len(molecules.mols) * 0.9)
training = molecules.mols[0:stride]
test = molecules.mols[stride:len(molecules.mols)]
print (len(molecules.mols), len(test), len(training))

1753 176 1577


In [6]:
training_dataset_info = bioalerts.LoadMolecules.GetDataSetInfo()
training_dataset_info.extract_substructure_information(radii=[2,3,4],mols=training)

In [7]:
print (len(training_dataset_info.substructure_dictionary.keys()))

16117


In [8]:
bioactivities = np.genfromtxt('tutorial/datasets/Young_2002.dat',
                              dtype='int',
                              skip_header=1,
                              usecols=1)
arr = np.arange(0,len(bioactivities))
mask = np.ones(arr.shape,dtype=bool)
mask[molecules.molserr]=0
bioactivities =  bioactivities[mask]

print (len(bioactivities))
print (len(molecules.mols))

1753
1753


In [9]:
bio_training = bioactivities[0:stride]
bio_test = bioactivities[stride:len(molecules.mols)]
print(len(bioactivities), len(bio_test), len(bio_training))

1753 176 1577


In [10]:
# Calcoliamo gli alerts
Alerts_categorical = bioalerts.Alerts.CalculatePvaluesCategorical(max_radius=4)

# Prediciamo i valori per il test set
Alerts_categorical.calculate_p_values(mols=test,
                                      substructure_dictionary=training_dataset_info.substructure_dictionary,
                                      bioactivities=bio_training,
                                      mols_ids=training_dataset_info.mols_ids[0:stride],
                                      threshold_nb_substructures = 5,
                                      threshold_pvalue = 0.05,
                                      threshold_frequency = 0.7,
                                      Bonferroni=True)

Number of substructures processed:  4723
Significant substructures:  71 substructures


In [11]:
print(len(Alerts_categorical.output))

71


In [14]:
Alerts_categorical.XlSXOutputWriter(Alerts_categorical.output, 
                                    'test_set_predictions_Young_2002.xlsx')